# 5.2 参数管理
- **目录**
  - 5.2.1 参数访问
    - 5.2.1.1 目标参数
    - 5.2.1.2 一次性访问所有参数
    - 5.2.1.3 从嵌套块收集参数
  - 5.2.2 参数初始化
    - 5.2.2.1 内置初始化
    - 5.2.2.2 自定义初始化
  - 5.2.3 参数绑定

- 在**选择了架构**并**设置了超参数**后，模型将进入**训练阶段**。
- 此时的目标是找到**使损失函数最小化的模型参数值。**
- 经过训练后，我们将需要使用这些参数来做出未来的预测。
- 此外，有时希望**提取参数**，以便在其他环境中**复用**它们，将模型保存下来，以便它可以在其他软件中执行，或者为了获得科学的理解而进行检查。
- 本节将介绍操作参数的具体细节，包括如下内容：
  * 访问参数，用于调试、诊断和可视化。
  * 参数初始化。
  * 在不同模型组件间共享参数。

In [1]:
%matplotlib inline
import torch
from torch import nn
##torch有对权重和偏置进行自动初始化的功能
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.4158],
        [0.4272]], grad_fn=<AddmmBackward0>)

## 5.2.1 参数访问
- 当通过`Sequential`类定义模型时，可以**通过索引来访问模型的任意层。**
- 这就像模型是一个列表一样，每层的参数都在其属性中。
- 如下所示可以检查第二个全连接层的参数。

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1722,  0.3258, -0.2437,  0.3485,  0.3230,  0.0465,  0.2103,  0.2977]])), ('bias', tensor([-0.0523]))])


In [3]:
list(net[2].parameters())

[Parameter containing:
 tensor([[ 0.1722,  0.3258, -0.2437,  0.3485,  0.3230,  0.0465,  0.2103,  0.2977]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0523], requires_grad=True)]

In [4]:
net[2].state_dict()['weight'].shape

torch.Size([1, 8])

----------
- **说明：序列模块前向传播的手工实现**

In [5]:
'''
序列模块前向传播的手工实现：
测试参数与输入值之间的计算关系
尤其是权重的形状有点令人困惑：很显然权重是按照行向量进行保存的
'''
from torch.nn import functional as F
w0=net[0].state_dict()['weight']
b0=net[0].state_dict()['bias']
o1=F.relu(X@w0.T+b0)
w2=net[2].state_dict()['weight']
b2=net[2].state_dict()['bias']
o2=o1@w2.T+b2
o2

tensor([[0.4158],
        [0.4272]])

In [6]:
##下述取值方式与上述方式一致
w0=net[0].weight.data
b0=net[0].bias.data
o1=F.relu(X@w0.T+b0)
w2=net[2].weight.data
b2=net[2].bias.data
o2=o1@w2.T+b2
o2

tensor([[0.4158],
        [0.4272]])

-----------

- 输出的结果告诉我们一些重要的事情：
  - 首先，这个**全连接层包含两个参数，分别是该层的权重和偏置。**
  - 两者都存储为单精度浮点数（float32）。
  - 注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

### 5.2.1.1 目标参数

- 注意，每个参数都表示为参数类的一个实例。
- 要对参数执行任何操作，首先我们需要访问底层的数值。
- 下面的代码**从第二个全连接层（即第三个神经网络层）提取偏置，
提取后返回的是一个参数类实例，并进一步访问该参数的值**。

In [7]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].state_dict()['bias']) ##也可以通过字典属性访问
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0523], requires_grad=True)
tensor([-0.0523])
tensor([-0.0523])


- 参数是复合的对象，包含**值**、**梯度**和**额外信息**。
这就是我们需要显式参数值的原因。
- 除了值之外，我们还可以访问每个参数的梯度。
在上面这个网络中，**由于我们还没有调用反向传播，所以参数的梯度处于初始状态。**


In [8]:
net[2].weight.grad == None

True

### 5.2.1.2 一次性访问所有参数

- 当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。
- 当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，
因为我们需要**递归整个树**来提取每个子块的参数。
- 下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。


In [9]:
## 访问第一个全连接层的参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
## 访问所有层的参数
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


- 另一种访问网络参数的方式，如下所示：

In [10]:
## 注意字典中的键
net.state_dict()['0.bias'], net.state_dict()['2.bias']

(tensor([-0.4384,  0.1339, -0.0263,  0.1567,  0.1603,  0.4208,  0.2504, -0.3191]),
 tensor([-0.0523]))

In [29]:
net.named_parameters

<bound method Module.named_parameters of Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)>

### 5.2.1.3 从嵌套块收集参数

- 如果将多个块相互嵌套，参数命名约定是如何工作的?
- 首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。


In [11]:
## block1中有两个全连接层
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())
## block2将block1嵌套4次，相当于8个全连接层
def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

## 最后一个输出层，输出一个标量作为结果
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3582],
        [0.3582]], grad_fn=<AddmmBackward0>)

- 设计了网络后，在看工作方式：


In [12]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


- 因为层是分层嵌套的，所以也可以像通过嵌套列表索引一样访问它们。
- 下面访问第一个主要的块中、第二个子块的第一层的偏置项。


In [14]:
## 取出某层的偏置参数的值
rgnet[0][1][0].bias.data

tensor([ 0.4556,  0.0797, -0.2463,  0.3337,  0.0246, -0.3351,  0.4367,  0.0798])

## 5.2.2 参数初始化

- 第4.8节中讨论了良好初始化的必要性。
- 深度学习框架提供**默认随机初始化**方法，也允许我们创建**自定义初始化方法**。
- 默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，这个范围是根据输入和输出维度计算出的。
- 除此之外，PyTorch的`nn.init`模块提供了多种预置初始化方法。

### 5.2.2.1 内置初始化
- 首先调用内置的初始化器。
- 下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，且将偏置参数设置为0。


In [15]:
def init_normal(m):
    if type(m) == nn.Linear:
        ## 使用正态分布随机数初始化权重
        nn.init.normal_(m.weight, mean=0, std=0.01)
        ## 偏置全部初始化为0
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0201, -0.0038, -0.0204, -0.0055]), tensor(0.))

- 还可以将所有参数初始化为给定的常数，比如初始化为1。


In [16]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

- 还可以对某些块应用不同的初始化方法：
  - 例如，下面我们使用Xavier初始化方法初始化第一个神经网络层。
  - 然后将第三个神经网络层初始化为常量值42。


In [17]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
'''
注意初始化参数一般是调用net的apply函数，以初始化函数作为参数
'''
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.2908, -0.3863,  0.2172, -0.0157])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [18]:
'''
nn.init包中的初始化函数列表
注意不带下划线的同名初始化函数将废除
'''
dir(nn.init)

['Tensor',
 '_Optional',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_calculate_correct_fan',
 '_calculate_fan_in_and_fan_out',
 '_make_deprecate',
 '_no_grad_fill_',
 '_no_grad_normal_',
 '_no_grad_trunc_normal_',
 '_no_grad_uniform_',
 '_no_grad_zero_',
 'calculate_gain',
 'constant',
 'constant_',
 'dirac',
 'dirac_',
 'eye',
 'eye_',
 'kaiming_normal',
 'kaiming_normal_',
 'kaiming_uniform',
 'kaiming_uniform_',
 'math',
 'normal',
 'normal_',
 'ones_',
 'orthogonal',
 'orthogonal_',
 'sparse',
 'sparse_',
 'torch',
 'trunc_normal_',
 'uniform',
 'uniform_',
 'warnings',
 'xavier_normal',
 'xavier_normal_',
 'xavier_uniform',
 'xavier_uniform_',
 'zeros_']

### 5.2.2.2 自定义初始化

- 有时，深度学习框架没有提供所需初始化方法。
在下面的例子中，使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned} \tag{5.2.1}
$$


- 同样，我们实现了一个`my_init`函数来应用到`net`：


In [19]:
def my_init(m):
    if type(m) == nn.Linear:
        '''
        此处显示自动初始化后的值,第一次运行时按照torch的规则进行自动初始化，使用了均匀分布。
        均匀分布函数使用的参数可以查看torch文档：
        https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear
        '''
        print("Init:", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        ## 然后再调用均匀分布函数后的初始值
        nn.init.uniform_(m.weight, -10, 10)
        ## 调整均匀分布的参数值，符合5.2.1的分段初始化
        m.weight.data *= m.weight.data.abs() >= 5
        print('>=5设置后:',m.weight.data)

In [20]:
net.apply(my_init)
net[0].weight[:2]

Init: weight torch.Size([8, 4])
>=5设置后: tensor([[-0.0000,  0.0000, -6.7173, -6.1001],
        [-0.0000,  0.0000,  5.8280,  8.7185],
        [-6.5796,  0.0000, -9.2288,  9.7044],
        [ 6.5312,  0.0000, -0.0000,  0.0000],
        [-0.0000, -9.9691, -5.9393, -0.0000],
        [ 0.0000,  0.0000,  8.8339,  5.7148],
        [ 0.0000, -0.0000,  0.0000,  0.0000],
        [ 7.9076, -9.9015,  7.7125,  0.0000]])
Init: weight torch.Size([1, 8])
>=5设置后: tensor([[-8.9822, -5.7516,  7.1305, -8.9823,  0.0000, -0.0000, -8.2990,  7.7230]])


tensor([[-0.0000,  0.0000, -6.7173, -6.1001],
        [-0.0000,  0.0000,  5.8280,  8.7185]], grad_fn=<SliceBackward0>)

In [21]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

- 注意，我们始终可以**直接设置参数**。


In [22]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -5.7173, -5.1001])

## 5.2.3 参数绑定

- 有时希望在多个层间共享参数：
  - 先定义一个稠密层。
  - 然后使用它的参数来设置另一个层的参数。


In [23]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),##是一个新层，与上面一层参数完全相同
                    shared, nn.ReLU(),##同上
                    nn.Linear(8, 1))
net(X)

tensor([[0.2879],
        [0.2987]], grad_fn=<AddmmBackward0>)

In [24]:
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


- 这个例子表明第三个和第五个神经网络层的**参数是绑定**的。
- 它们不仅值相等，而且由相同的张量表示。
- 因此，如果我们改变其中一个参数，另一个参数也会改变。
- 当参数绑定时，梯度会发生什么情况？
  - 答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层即第三个神经网络层和第三个隐藏层即第五个神经网络层的梯度会加在一起。


- **说明：共享参数**
  - 上述代码将`shared`层（即nn.Linear(8, 8)）添加到神经网络中作为第三个和第五个神经网络层。由于这两个层实际上引用的是同一个`nn.Linear`对象，因此它们共享相同的权重和偏置参数。
  - 当进行反向传播计算梯度时，梯度会在每一层上传递，并更新与之相关的权重参数。
    - 对于这个特定的示例，第三个神经网络层（第二个隐藏层）的梯度与第五个神经网络层（第三个隐藏层）的梯度实际上是针对相同的权重和偏置参数计算的。
    - 因此，它们的梯度会被累加在一起以更新这些共享参数。
  - 这里的关键点是，由于`shared`层在多个位置使用（第三个和第五个神经网络层），所以在反向传播过程中，梯度会叠加，从而同时影响这两个位置的参数更新。
  - 这意味着在训练期间，这两个层的参数始终保持相同，因为它们实际上是**同一个`nn.Linear`对象的实例**。

In [25]:
## 第3和第5个神经网络层实际上是同一个对象
id(net[2]), id(net[4])

(1950369138832, 1950369138832)

In [26]:
## 激活层虽然也是一个层，但是其参数为空
list(net[1].named_parameters())

[]

## 小结

* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。